In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#### stage one model - quantity prediction

In [31]:
def stage1_pred(data, input_cols, prods):
    #input_cols = ['OrderQuantity_lag1','Temperature', 'Rainfall', 'CPI', 'Female','Age','Income',]
    stage1_models = {}
    tmp = []
    for prod in prods:
        data_tmp = data[data['ProductID']==prod]
        print(data_tmp.shape, data.shape)
        X = data_tmp[input_cols]
        X = sm.add_constant(X)
        y = data_tmp['OrderQuantity']
        model = sm.OLS(y, X).fit()
        stage1_models[prod]=model
        predictions = model.predict(X)
        data_tmp['pred_s1'] = predictions
        tmp.append(data_tmp)
    data = pd.concat(tmp)
    return stage1_models, data

data = pd.read_csv('./data/stage1_data.csv')
## drop rows with missing records
data = data.dropna()
input_cols = ['OrderQuantity_lag1','Temperature', 'Rainfall', 'CPI', 'Female','Age','Income',]
stage1_models, data = stage1_pred(data, input_cols, [1,2,3,4])

(17900, 15) (71600, 15)
(17900, 15) (71600, 15)
(17900, 15) (71600, 15)
(17900, 15) (71600, 15)


In [32]:
for prod in [1,2,3,4]:
    print(stage1_models[prod].summary())

                            OLS Regression Results                            
Dep. Variable:          OrderQuantity   R-squared:                       0.536
Model:                            OLS   Adj. R-squared:                  0.536
Method:                 Least Squares   F-statistic:                     2952.
Date:                Sun, 16 Jun 2024   Prob (F-statistic):               0.00
Time:                        19:14:25   Log-Likelihood:                 12376.
No. Observations:               17900   AIC:                        -2.474e+04
Df Residuals:                   17892   BIC:                        -2.467e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.3938      0

#### stage two model - membership likehood prediction